In [ ]:
import numpy as np

## Método de Relaxação

In [ ]:
def relax(x1, c):
    f = lambda x, c: 1 - np.exp(-c*x)
    g = lambda x, c: c*np.exp(-c*x2)
    epsilon = 1
    i = 0
    while epsilon > 1e-6:
        x1, x2 = f(x1,c), x1
        epsilon = np.abs((x1-x2) / (1 - 1/(g(x2,c))))
        i+=1
    print("Aproximação:", x1, "       Número de iterações:",i,"\n")

In [ ]:
relax(1,2)

## Método de Sobre-Relaxação

In [ ]:
def overrelax(w,c):
    f = lambda x, c: 1 - np.exp(-c*x)
    g = lambda x, c: c*np.exp(-c*x2)
    x2= 1
    x1 = x2 + (1 + w)*(f(x2,2)-x2)
    epsilon = 1
    i = 0
    while epsilon > 1e-6:
        x1, x2 = (1 + w)*f(x1,c) - w*x1, x1
        epsilon = np.abs((x1-x2) / (1 - 1/((1+w)*g(x2,c)-w)))
        i+=1
    print("Aproximação:", x1, "       Número de iterações:",i,"\n")

In [ ]:
overrelax(0.7,2)

## Método da Bisseção

In [ ]:
def binary(x1, x2, Erro):
    while np.abs(x1-x2) > Erro:
        if f(x1)*f(x2)<0:
            x = 0.5 * (x1+x2)
            if f(x1)*f(x)<0:
                x2 = x
            elif f(x1)*f(x2)==0:
                    return x
            else:
                x1 = x
        else:
            return "Valores não se enquadram"
    return x

In [ ]:
binary(-2,5,1e-4)

## Método de Newton

### 1 dimensão

In [ ]:
def P(x):
    return 924*x**6 - 2772*x**5 + 3150*x**4 - 1680*x**3 + 420*x**2 - 42*x + 1

def dP(x):
    return 6*924*x**5 - 5*2772*x**4 + 4*3150*x**3 - 3*1680*x**2 + 2*420*x - 42

def Newton(x0, Erro):
    x=x0-P(x0)/dP(x0)
    i = 0
    while abs(x-x0)>Erro:
        x,x0 = x-P(x)/dP(x),x
        i += 1
    return x, i

In [ ]:
print("x0 =", Newton(0.1, 1e-10)[0])
print("x1 =", Newton(0.2, 1e-10)[0])
print("x2 =", Newton(0.4, 1e-10)[0])
print("x3 =", Newton(0.7, 1e-10)[0])
print("x4 =", Newton(0.9, 1e-10)[0])
print("x5 =", Newton(1, 1e-10)[0])

### 2 ou mais dimensões

In [ ]:
Vmais = 5
R1 = 1e3
R2 = 4e3
R3 = 3e3
R4 = 2e3
I0 = 3e-9
VT = 0.05

def f_V1(V1,V2):                            
    return ((V1-Vmais)/R1) + (V1/R2) + I0*(np.exp((V1-V2)/VT)-1)  

def f_V2(V1,V2):                            
    return ((V2-Vmais)/R3) + (V2/R4) - I0*(np.exp((V1-V2)/VT)-1)

def derivada_V1_V1(V1,V2):                  # derivada de f(V1) em ordem a V1 
    return (1/R1) + (1/R2) + (I0/VT)*(np.exp((V1-V2)/VT))

def derivada_V1_V2(V1,V2):                  # derivada de f(V1) em ordem a V2 
    return -(I0/VT)*(np.exp((V1-V2)/VT))
 
def derivada_V2_V2(V1,V2):                  # derivada de f(V2) em ordem a V2 
    return (1/R3) + (1/R4) + (I0/VT)*(np.exp((V1-V2)/VT))

def derivada_V2_V1(V1,V2):                  # derivada de f(V2) em ordem a V1 
    return -(I0/VT)*(np.exp((V1-V2)/VT))


def Jacobiano(V1,V2):
    J = np.array([[derivada_V1_V1(V1,V2),derivada_V1_V2(V1,V2)],
                  [derivada_V2_V1(V1,V2),derivada_V2_V2(V1,V2)]])
    return J

def B(V1,V2):
    b1 = np.array([f_V1(V1,V2),f_V2(V1,V2)])      
    return b1

def solveGaussPivotParcial(A, B):
    N = len(B)
    linhas = np.arange(N) 
    sol = 0 * B

    for p in range(N):
        index = p + np.argmax(np.abs(A[linhas[p:], p]))
        linhas[p], linhas[index] = linhas[index], linhas[p]

        for i in range(p+1,N): 
            B[linhas[i]] -= A[linhas[i] , p] * B[linhas[p]] / A[linhas[p],p]
            A[linhas[i],p:] -= A[linhas[i] , p] * A[linhas[p],p:] / A[linhas[p],p]

    for i in range(N-1,-1,-1):
        sol[i] = (B[linhas[i]] - np.dot(A[linhas[i],i+1:] , sol[i+1:])) / A[linhas[i],i]

    return sol

delta_x = np.array([1,1],float)
x = np.array([10,10],float)

while np.linalg.norm(delta_x) > 1e-6:
    V1,V2=x[0],x[1]
    delta_x = solveGaussPivotParcial(Jacobiano(V1,V2),B(V1,V2))                           
    x -= delta_x

print('[V1 , V2] = ', x, '(V)','\n')

## Método da secante

In [ ]:
def secant(a, b, Erro):
    f = lambda x: x**2-2
    epsilon = 1
    i = 0
    while epsilon > Erro:
        i += 1
        b, a = b - f(b) * ((b - a) / (f(b) - f(a))), b                           
        epsilon = np.abs(b - a)  
        print(a)
    return a, i

In [ ]:
secant(-2,-4,1e-10)

## Método da Secção Áurea

In [ ]:
def f(x):
    return (x**3)/(np.exp(x)-1)

def aurea(func, x1, x4, ERRO):
    z = (1 + np.sqrt(5))/2
    x2 = x4 - (x4-x1)/z
    x3 = x1 + (x4-x1)/z
    f1, f2, f3, f4 = func(x1), func(x2), func(x3), func(x4)

    while abs(x4-x1)>ERRO:
        if f2<f3:
            x4,f4 = x3,f3
            x3,f3 = x2,f2
            x2 = x4 - (x4-x1)/z
            f2 = func(x2)
        else:
            x1,f1 = x2,f2
            x2,f2 = x3,f3
            x3 = x1 + (x4-x1)/z
            f3 = func(x3)

    return 0.5*(x1+x4)


In [ ]:
aurea(f,-2, 2, 1e-10)

## Método de Gradiente

In [ ]:
def f(x):
    return x**4 - x**3   # Função a usar

def derivada_1(x):
    return 4*x**3 + 3*x**2    # Derivada da função a usar

def derivada_2(x):
    return 12 * x**2 + 12*x      # 2ª derivada da função a usar

# Método do Gradiente
gama = 0.01            # o valor de gama tem que ser pequeno porque senão a iteração vai para +infinito e nunca vai convergir 
count = 0
x0 = -1
while (np.abs(derivada_1(x0)) > 1e-10 and count < 1000 and abs(f(x0)) < 1000):    # não queremos mais de 1000 iterações
    x0 = x0 - gama * derivada_1(x0)   
    count += 1
print(x0, derivada_1(x0))
